In [ ]:
using StatsBase
include("./algoritmos.jl")

In [ ]:
# referência de ordenação
function ref(X::Vector{Int})
    v = sort(X)
end

In [ ]:
# CLRS page 112 max_heapify
# T(h) <= T(h-1) + theta(1)
# T(h) = O(h) ~ O( lg(m) )
function desce_heap(A::Vector{Int}, m::Int, i::Int)
    # println(A)
    e = 2i
    d = 2i+1
    maior = ( ( e <= m && A[e] > A[i] ) ? e : i )
    if d <= m && A[d] > A[maior]   maior = d   end
    if maior != i
        swap(A, i, maior)
        desce_heap(A, m, maior)
    end
end

In [ ]:
# teste desce_heap
vec = [13, 46, 17, 34, 41, 15, 14, 23, 30, 21, 10, 12]
A = copy(vec)
m = size(A)[1]

desce_heap(A, m, 1)

res = [46, 41, 17, 34, 21, 15, 14, 23, 30, 13, 10, 12]
println("desce_heap aprovado: ", A == res)

In [ ]:
# CLRS page 114
function build_max_heap(A::Vector{Int}, n::Int)
    med = medians(1, n)[1]   # floor n/2
    for i in med : -1 : 1
        desce_heap(A, n, i)
    end
end

In [ ]:
# teste build_max_heap
vec = [14, 13, 34, 17, 15, 10, 46, 23, 12, 41, 30, 21]
A = copy(vec)
n = size(A)[1]

build_max_heap(A, n)

res = [46, 41, 34, 23, 30, 21, 14, 17, 12, 15, 13, 10]
println("build_max_heap aprovado: ", A == res)

In [ ]:
# 6.4 Algoritmo de ordenação por heap, CLRS pag 116
# ordenação crescente
function heapsort(A, n)
    build_max_heap(A, n)             # pré-processamento: cria um max heap
    heapSize = copy(n)               # controle de processamento dos elementos do heap
    for i in n : -1 : 2    
        swap(A, 1, i)                # max_heap = último elemento da lista
        heapSize -= 1
        desce_heap(A, heapSize, 1)   # heap desfeito pelo swap, então refaz o heap entre 1 : heapSize
    end
end

In [ ]:
# teste heapsort
nTestes = 1000000
ok = []
for i in 1 : nTestes
    N = rand(1:1000)
    A = sample(1:2N, N, replace = false); X = copy(A)

    heapsort(X, N)

    push!( ok, X == ref(A) )
end
println("heapsort aprovado: ", sum(ok) == nTestes)